1. Separate the Data 


Survey Data: This is your baseline. It represents what voters said their preferences were.
Election Results: Separate the electronic votes from the polling station votes to analyze each type of voting independently.

Let's load the data.

In [22]:
import pandas as pd

private_data = pd.read_excel(r'data/new/synthetic_dataD.xlsx')
public_data_register = pd.read_excel(r'data/old/public_data_registerD.xlsx')
public_data_results = pd.read_excel(r'data/old/public_data_resultsD.xlsx')


2. Organize Data for Comparison

Create tables showing the percentage or count of votes for each party in:

The survey data

The election results for electronic votes

The election results for polling station votes



In [23]:
private_totals = private_data['party'].value_counts()

# Get the total amount of votes
public_data_totals = public_data_results.tail(2)

# Rename the index to match the private data
public_data_totals.set_index('Unnamed: 0', inplace=True)






# Calculate the amount of paper votes
public_data_totals.loc['Paper'] = public_data_totals.loc['Total'].apply(int) - public_data_totals.loc['E-votes'].apply(int)

# Ensure private_totals is a DataFrame and transpose it to match the public data
private_totals = private_totals.to_frame().transpose()

print(private_totals)
print('\n')




public_data_totals.rename(columns={

    'Invalid ballots': 'Invalid vote'
}, inplace=True)

public_data_totals.rename(index={'E-votes':'electronic','Total_row':'count', 'Paper':'paper'}, inplace=True)

# Drop the 'Unnamed: 0' row when reading the data
#public_data_totals.drop(index='Unnamed: 0')

# Rename the index to match the private data
#public_data_totals.index = ['electronic','count', 'paper']

print(public_data_totals)
print('\n')
print(public_data_totals.index)

party  Green  Red  Invalid vote
count    181  113             6


            Red  Green  Invalid vote  Total
Unnamed: 0                                 
electronic  107    199             7    313
Total       386    630            20   1036
paper       279    431            13    723


Index(['electronic', 'Total', 'paper'], dtype='object', name='Unnamed: 0')


C:\Users\skrbj\AppData\Local\Temp\ipykernel_6948\3118264266.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_data_totals.loc['Paper'] = public_data_totals.loc['Total'].apply(int) - public_data_totals.loc['E-votes'].apply(int)
C:\Users\skrbj\AppData\Local\Temp\ipykernel_6948\3118264266.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_data_totals.rename(columns={
C:\Users\skrbj\AppData\Local\Temp\ipykernel_6948\3118264266.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [24]:
# Initialize the private_totals DataFrame with party names
private_totals = pd.DataFrame(index=['paper', 'electronic'], columns=['Green', 'Red', 'Invalid vote'])

# Calculate counts for electronic votes (evote == 1)
for party in ['Green', 'Red', 'Invalid vote']:
    private_totals.loc['electronic', party] = private_data.loc[(private_data['evote'] == 1) & (private_data['party'] == party), 'party'].count()

# Calculate counts for paper votes (evote == 0)
for party in ['Green', 'Red', 'Invalid vote']:
    private_totals.loc['paper', party] = private_data.loc[(private_data['evote'] == 0) & (private_data['party'] == party), 'party'].count()

# Add a count row for totals
private_totals.loc['count'] = private_totals.sum()

# Add a total row for each party
private_totals['Total'] = private_totals.sum(axis=1)

# Display the resulting DataFrame
print(private_totals)

           Green  Red Invalid vote Total
paper        123   76            5   204
electronic    58   37            1    96
count        181  113            6   300



3. Choose a Statistical Test

Since you’re comparing categorical data (party preference distributions), a Chi-Square test of independence is a good option.
Chi-Square Test: This test will help you see if there are statistically significant differences in political preferences between:
Survey vs. Electronic Votes
Survey vs. Polling Station Votes
Each of these comparisons will show if the preferences in the survey align with the actual votes or if there are notable differences.


In [25]:
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind

# Create a contingency table for the Chi-Square test
contingency_table = pd.concat([private_totals.loc[['paper', 'electronic'], ['Green', 'Red', 'Invalid vote']],
                               public_data_totals.loc[['paper', 'electronic'], ['Green', 'Red', 'Invalid vote']]])

# Perform the Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-Value: {p}")

# Perform t-test for each party
for party in ['Green', 'Red', 'Invalid vote']:
    t_stat, p_val = ttest_ind(private_totals.loc[['paper', 'electronic'], party].astype(int),
                              public_data_totals.loc[['paper', 'electronic'], party].astype(int))
    print(f"T-Test for {party}:")
    print(f"T-Statistic: {t_stat}")
    print(f"P-Value: {p_val}\n")

Chi-Square Statistic: 2.6360064238125998
P-Value: 0.8529466453985117
T-Test for Green:
T-Statistic: -1.863583985329083
P-Value: 0.20340345750551414

T-Test for Red:
T-Statistic: -1.5479165519576517
P-Value: 0.2617279153660449

T-Test for Invalid vote:
T-Statistic: -1.9414506867883021
P-Value: 0.19170962313452386





4. Interpret the Results

If the p-value from your Chi-Square test is less than 0.05, it means there’s a statistically significant difference in preferences between the groups.
If there’s a significant difference, it could indicate a possible manipulation or other factors influencing the results.

### Interpretation of Results

#### Chi-Square Test
- **Chi-Square Statistic**: 6.360299944406317
- **P-Value**: 0.3840618947439205

The p-value from the Chi-Square test is greater than 0.05, indicating that there is no statistically significant difference in preferences between the groups.

#### T-Tests
- **T-Test for Green**:
    - **T-Statistic**: -2.1694171309207846
    - **P-Value**: 0.16227922122515373

- **T-Test for Red**:
    - **T-Statistic**: -1.7757517924317536
    - **P-Value**: 0.2177604255759461

- **T-Test for Invalid vote**:
    - **T-Statistic**: -2.5298221281347035
    - **P-Value**: 0.12712843905603044

All p-values from the T-Tests are greater than 0.05, indicating that there are no statistically significant differences in the counts for each party between the survey data and the election results.

### Conclusion
Based on the Chi-Square test and T-Tests, there are no statistically significant differences in political preferences between the survey data and the election results. This suggests that the survey results align with the actual votes, and there is no evidence of manipulation or other factors influencing the results.

Private Data:
Chi-Square Statistic: 6.360299944406317
P-Value: 0.3840618947439205
T-Test for Green:
T-Statistic: -2.1694171309207846
P-Value: 0.16227922122515373

T-Test for Red:
T-Statistic: -1.7757517924317536
P-Value: 0.2177604255759461

T-Test for Invalid vote:
T-Statistic: -2.5298221281347035
P-Value: 0.12712843905603044


Synthetic data:
Chi-Square Statistic: 2.6360064238125998
P-Value: 0.8529466453985117
T-Test for Green:
T-Statistic: -1.863583985329083
P-Value: 0.20340345750551414

T-Test for Red:
T-Statistic: -1.5479165519576517
P-Value: 0.2617279153660449

T-Test for Invalid vote:
T-Statistic: -1.9414506867883021
P-Value: 0.19170962313452386